In [1]:
import pandas as pd

data = pd.read_csv('data_sample_three_years.csv')

data = data.iloc[:, [0,1,2,3,4]]
data.rename(columns={'date': 'open', 'low':'close', 'open':'high', 'high':'low'}, inplace=True)

data['date'] = pd.to_datetime(data['timestamp'], unit='s')
data = data.iloc[:, [0,5,1,2,3,4]]

data_backup = data.loc[ data.timestamp >= 1675814400 ]
data = data.loc[ data.timestamp < 1675814400 ]

In [2]:
seq_length = 10  # Number of time steps in each sequence
close_prices = data.close

In [3]:
from sklearn.preprocessing import MinMaxScaler

# Scaling the 'close' prices
scaler = MinMaxScaler()
data['close_scaled'] = scaler.fit_transform(data[['close']])

In [4]:
import numpy as np

# Create input sequences and targets
def create_sequences(data, seq_length):
    X = []
    y = []
    for i in range(len(data) - seq_length):
        X.append(data.iloc[i:i+seq_length])  # Input sequence
        y.append(data.iloc[i+seq_length])    # Target value (next data point)
    return np.array(X), np.array(y)

In [5]:
X, y = create_sequences(data[['close_scaled']], seq_length)
X = X.reshape(X.shape[0], seq_length, 1)

In [6]:
NEURONS = [25, 50, 100]
OPTIMIZERS = ['adam', 'rmsprop']
BATCH_SIZES = [25, 32, 45]

In [7]:
import itertools

# Generate all combinations of parameters
param_combinations = list(itertools.product(NEURONS, OPTIMIZERS, BATCH_SIZES))
total_combinations = len(param_combinations)
print(f"param_combinations: {total_combinations} different options.")

param_combinations: 18 different options.


In [8]:
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error, accuracy_score

best_features = None
best_score = float('inf')

for i, params in enumerate(param_combinations):
    neurons = params[0]
    optimizer = params[1]
    batch_size = params[2]

    # Build LSTM model
    model = Sequential([
        LSTM(neurons, input_shape=(seq_length, 1)),
        Dense(1)
    ])
    
    # Compile the model
    model.compile(optimizer=optimizer, loss='mean_squared_error')
    
    # Adding early stopping to prevent overfitting
    early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
    
    # Training the model with early stopping
    model.fit(X, y, epochs=100, batch_size=batch_size, validation_split=0.2, callbacks=[early_stopping])

    db = data_backup.copy()
    db['close_scaled'] = scaler.transform(db[['close']])
    X_b, y_b = create_sequences(db['close_scaled'], seq_length)
    X_b = X_b.reshape(X_b.shape[0], seq_length, 1)

    y_pred = model.predict(X_b)

    y_pred = scaler.inverse_transform(y_pred)

    score = mean_squared_error(y_pred, db[['close']].iloc[10:])
    print(f'MSE for {params} is {score}')
    with open('lstm-3_2-log.txt', 'a+') as file:
        file.write(f'MSE for {params} is {score}\n')

    # Update best feature combination if score improves
    if score < best_score:
        best_score = score
        best_features = params

print("Best feature combination:", best_features)
print("Best MSE:", best_score)

2024-03-25 08:49:48.061714: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-25 08:49:48.255883: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 08:49:48.256002: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 08:49:48.259823: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-03-25 08:49:48.318461: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-25 08:49:50.503967: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

Epoch 1/100
521/521 [==============================] - 8s 9ms/step - loss: 0.0050 - val_loss: 1.5677e-04
Epoch 2/100
521/521 [==============================] - 4s 8ms/step - loss: 1.8904e-04 - val_loss: 4.4099e-05
Epoch 3/100
521/521 [==============================] - 4s 7ms/step - loss: 1.7652e-04 - val_loss: 3.5725e-05
Epoch 4/100
521/521 [==============================] - 4s 7ms/step - loss: 1.6921e-04 - val_loss: 4.0054e-05
Epoch 5/100
521/521 [==============================] - 4s 7ms/step - loss: 1.5961e-04 - val_loss: 3.0004e-05
Epoch 6/100
521/521 [==============================] - 4s 7ms/step - loss: 1.4898e-04 - val_loss: 2.9561e-05
Epoch 7/100
521/521 [==============================] - 3s 6ms/step - loss: 1.3533e-04 - val_loss: 2.0409e-05
Epoch 8/100
521/521 [==============================] - 4s 7ms/step - loss: 1.2605e-04 - val_loss: 2.0063e-05
Epoch 9/100
521/521 [==============================] - 4s 7ms/step - loss: 1.1879e-04 - val_loss: 1.5758e-05
Epoch 10/100
521/521 [=